## 雷达仿真

通过FMCW雷达仿真代码，比较各类DOA算法性能。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 设计雷达系统参数
f0_,c_ = 77e9, 3e8 
waveL = c_ / f0_ # 波长

r_res, v_res = 0.3, 0.2

B = c_ / (2*r_res)      # 带宽
print(f"带宽B: {B * 1e-6} MHz")

N_chirp = 256
T_chirp = waveL / (2*v_res*N_chirp)
T_frame = N_chirp * T_chirp
maxV = waveL / (4*T_chirp)  # Chirp时长

print(f"T_chirp: {T_chirp*1e3}, T_frame: {T_frame*1e3}, maxV: {maxV}")

ADC_smp = 256
fs = ADC_smp / T_chirp
s_ = B / T_chirp
maxR_ = c_ * fs * 0.5 / (2 * s_) 

print(f"fs: {fs * 1e-6}MHz, maxR_: {maxR_}m")

RX_N = 12
d = 0.5 * waveL 
D = np.arange(RX_N) * d  # 接收天线阵列

R_FFT_N, V_FFT_N = 256, 256 #距离FFT和速度FFT

In [ ]:
# 回波信号仿真

tar_num = 2
R = np.array([10, 15])  # 目标距离
v = np.array([12, -10])  # 目标速度
theta = np.array([20, 25]) / 180.0 * np.pi   # 目标角度

rx_sig1 = np.zeros((len(R), RX_N, N_chirp, ADC_smp), dtype=complex)

for m in range(len(R)):
    for k in range(N_chirp):
        for L in range(RX_N):
            for n in range(ADC_smp):
                rx_sig1[m, L, k, n] = np.exp(1j * 2 * np.pi * (
                    (0.5 * B / T_chirp * 2 * ((2 * (R[m] + v[m] * k * T_chirp)) / c_ - D[L] * np.sin(theta[m]) / c_)) * T_chirp / ADC_smp * n +
                    f0_ * (2 * (R[m] + v[m] * k * T_chirp) / c_ - D[L] * np.sin(theta[m]) / c_)
                ))
                
data_in = np.sum(rx_sig1, axis=0)

In [ ]:
# # 选择一个接收器和一个Chirp来可视化
# L = 0
# k = 0
# plt.figure(figsize=(10,4))
# plt.plot(np.real(data_in[L, k, :]), label='Real Part')
# # plt.plot(np.imag(data_in[L, k, :]), label='Imaginary Part')
# plt.title('Received Signal at Receiver {} and Chirp {}'.format(L, k))
# plt.xlabel('Sample Index')
# plt.ylabel('Amplitude')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
from scipy.signal import hamming, get_window

# 生成窗函数
window = hamming(R_FFT_N)  # 汉明窗

data_cube = np.zeros([RX_N, N_chirp, R_FFT_N], dtype=complex)

# range FFT
for L in range(RX_N):
    for i in range(N_chirp):
        data_cube[L, i, :] = np.fft.fft(data_in[L, i, :] * window, R_FFT_N)
        # plt.plot(np.abs(data_cube[L, i, :]))

# 速度维FFT
for L in range(RX_N):
    for i in range(R_FFT_N):
         data_cube[L, :, i] = np.fft.fftshift(np.fft.fft(data_cube[L, :, i], N_chirp))

# # 距离维FFT
# data_cube = np.fft.fft(data_in, n=R_FFT_N, axis=2)
# # plt.figure()
# # plt.plot(np.abs(data_cube[0,0,0:30]),'-o')

# # 速度维FFT
# data_cube = np.fft.fft(data_cube, n=V_FFT_N, axis=1)
# data_cube = np.fft.fftshift(data_cube, axes=1)  # FFT shift for alignment

# 计算距离和速度坐标轴
rangeBin =  (c_ * T_chirp / (2*B)) * (fs / R_FFT_N)
range_axis = np.linspace(0, int(R_FFT_N/2)) * rangeBin 
velBin = (c_ / f0_ * 0.5) / (T_chirp * V_FFT_N)  
velocity_axis = np.linspace(-V_FFT_N / 2 + 1, V_FFT_N / 2, V_FFT_N) * velBin

plt.figure()
plt.imshow((20 * np.log10(np.abs(data_cube[L, :, 0:int(R_FFT_N/2)]))), aspect='auto', extent=(range_axis[0], range_axis[-1], velocity_axis[-1], velocity_axis[0]))
plt.colorbar()

在进行角度结算前，在回顾下雷达系统中的相位差知识。

### 相位差计算公式

相位差$\Delta \phi$ 的计算公式依赖于以下几个参数：

- $ d $：天线间的距离（天线阵列中相邻天线之间的距离）。
- $ \theta $：目标的方位角度。
- $ \lambda $：信号的波长。

相位差的基本公式是：

$$
\Delta \phi = \frac{2\pi}{\lambda} \Delta x
$$

其中，$\Delta x$ 是由于天线间距和目标方位角产生的路径长度差，计算方法为：

$$
\Delta x = d \sin(\theta)
$$

将 $\Delta x$ 代入相位差的公式中，我们得到：

$$
\Delta \phi = \frac{2\pi}{\lambda} (d \sin(\theta))
$$

### 详细解释

- **波长 ($\lambda$)**：这是雷达使用的信号的波长，它与雷达操作的频率有关。例如，77 GHz 的毫米波雷达的波长约为 3.9 毫米。
- **天线间距 ($d$)**：这是阵列中相邻天线之间的距离。通常，这个距离可以是波长的一半或其他分数，这取决于天线阵列的设计和所需的波束形成特性。
- **方位角 ($\theta$)**：这是目标相对于雷达阵列正前方的角度。角度 $ \theta $ 的正弦值 $\sin(\theta)$ 决定了由于天线间距引起的路径长度差。

这个相位差是关键参数，因为它直接影响到从雷达信号中提取目标位置信息的能力。通过测量不同天线接收到的信号的相位差，可以使用信号处理技术（如FFT）来估计目标的方位角。


In [ ]:
# using FFT
def doa_FFT(received_signals, fft_n):
    fft_data = np.abs(np.fft.fftshift(np.fft.fft(received_signals, fft_n))) # 角度维FFT
    fft_spectrum = 10 * np.log10(fft_data / np.max(fft_data))
    search_angles = np.linspace(-90, 90, fft_n)
    return search_angles, fft_spectrum 


# using MUSIC
def music_algorithm(received_signals, num_sources, num_antennas, n):
    """ MUSIC算法实现 """
    # 计算协方差矩阵
    R = received_signals @ received_signals.conj().T
    R /= np.trace(R)
    
    # 特征值分解
    eigenvalues, eigenvectors = np.linalg.eigh(R)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # 噪声子空间
    noise_subspace = eigenvectors[:, num_sources:]
    
    # 搜索角度
    search_angles = np.linspace(-90, 90, n)
    search_rad = np.deg2rad(search_angles)
    music_spectrum = np.zeros(search_angles.shape)
    
    for i, angle in enumerate(search_rad):
        steering_vector = np.exp(2j * np.pi * 0.5 * np.arange(num_antennas) * np.sin(angle))
        music_spectrum[i] = 1 / np.abs(steering_vector.conj().T @ noise_subspace @ noise_subspace.conj().T @ steering_vector)
    
    music_spectrum = 10 * np.log10(music_spectrum / np.max(music_spectrum))  # 转换为dB
    return search_angles, music_spectrum

In [ ]:
import numpy as np

def os_cfar_2d(data, guard_cells, noise_cells, gate, threshold_scale):
    """
    2D OS-CFAR detector.
    
    参数:
    - data: 输入的二维数组（雷达数据）
    - guard_cells: 保护单元的大小（元组形式，(行数, 列数)）
    - noise_cells: 噪声单元的大小（元组形式，(行数, 列数)）
    - threshold_scale: 用于调整阈值的缩放因子
    
    返回:
    - detections: 检测到的目标列表，每个元素为 (行, 列)
    """
    num_rows, num_cols = data.shape
    guard_rows, guard_cols = guard_cells
    noise_rows, noise_cols = noise_cells
    
    detections = []

    for row in range(num_rows):
        for col in range(num_cols):
            # 确定噪声和保护区域的边界
            row_start = max(row - guard_rows - noise_rows, 0)
            row_end = min(row + guard_rows + noise_rows + 1, num_rows)
            col_start = max(col - guard_cols - noise_cols, 0)
            col_end = min(col + guard_cols + noise_cols + 1, num_cols)
            
            # 保护区域边界
            guard_row_start = max(row - guard_rows, 0)
            guard_row_end = min(row + guard_rows + 1, num_rows)
            guard_col_start = max(col - guard_cols, 0)
            guard_col_end = min(col + guard_cols + 1, num_cols)
            
            # 提取噪声单元
            noise_patch = data[row_start:row_end, col_start:col_end]
            guard_patch = data[guard_row_start:guard_row_end, guard_col_start:guard_col_end]
            noise_patch = np.ma.array(noise_patch, mask=False)
            noise_patch.mask[guard_row_start-row_start:guard_row_end-row_start, guard_col_start-col_start:guard_col_end-col_start] = True
            
            # 计算阈值
            noise_level = np.mean(noise_patch.compressed())  # 使用压缩以去除被掩码的元素
            threshold = noise_level * threshold_scale
            
            # 检测
            if  data[row, col] > gate and data[row, col] > threshold and data[row, col] == np.max(guard_patch):
                print(data[row, col])
                detections.append((row, col))
    
    return detections

In [ ]:
# DOA估计
# 在进行DOA估计之前，需要先提取目标RangeBin对应的不同接收天线下的数据
data = 20 * np.log10(np.abs(data_cube[0, :, 0:int(R_FFT_N/2)]))

# plt.figure()
# plt.imshow(data)

guard_cells = (2, 2)
noise_cells = (4, 4)
gate, threshold_scale = 60, 2

detections = os_cfar_2d(data, guard_cells, noise_cells, gate, threshold_scale)
# print("检测到的目标位置:", detections)

for idx, det in enumerate(detections):
    
    rangeBin = det[1]
    
    # using FFT 
    FFT_num = 128
    # 使用MUSIC算法进行角度估计
    tar1_rx_data = data_cube[:, :,rangeBin] 
    print(tar1_rx_data.shape)

    search_angles, fft_spectrum = doa_FFT(data_cube[:,0,rangeBin], FFT_num)
    search_angles, music_spectrum = music_algorithm(tar1_rx_data.reshape([RX_N, -1]), 1, RX_N, FFT_num)

    # 绘制MUSIC谱
    plt.figure()
    plt.plot(search_angles, music_spectrum, '-ro')
    plt.plot(search_angles, fft_spectrum, '-bo')
    plt.xlabel('Angle (degrees)')
    plt.ylabel('Spectrum (dB)')
    plt.legend(['MUSIC', 'FFT'])
    plt.grid(True)
    plt.show()

## MUSIC

MUSIC（Multiple Signal Classification）算法是一种广泛用于信号处理领域的谱估计方法，特别是在阵列信号处理（如方向估计或源定位）中非常有效。MUSIC算法基于信号子空间和噪声子空间的概念，利用接收到的信号中的空间特性来估计波达方向（Direction of Arrival, DOA）或源位置。

### MUSIC算法的基本原理包括以下几个关键步骤：

#### 1. 数据模型
假设一个由$P$个窄带远场信号源构成的信号被一个包含$M$个传感器的阵列接收。信号模型可以表示为：
$\mathbf{X}(t) = \mathbf{A}(\mathbf{s}) \mathbf{s}(t) + \mathbf{n}(t)$
其中，$\mathbf{X}(t)$是接收信号矢量，$\mathbf{A}(\mathbf{s})$是由信号源的方向向量构成的阵列流形矩阵，$\mathbf{s}(t)$是源信号矢量，$\mathbf{n}(t)$是噪声矢量。

#### 2. 协方差矩阵的估计
计算接收信号的协方差矩阵：
$\mathbf{R} = \frac{1}{N} \sum_{t=1}^N \mathbf{X}(t) \mathbf{X}^H(t)$
其中，$N$是样本数，$H$表示共轭转置。

#### 3. 特征分解
对协方差矩阵$\mathbf{R}$进行特征分解，得到其特征值和特征向量。将特征向量按对应特征值的大小排序，最大的$P$个特征值对应的特征向量构成信号子空间，其余的特征向量构成噪声子空间。

#### 4. 空间谱函数的构建
MUSIC算法的核心在于构建一个空间谱函数，该函数在源存在的方向上具有峰值。空间谱函数定义为：
$P(\theta) = \frac{1}{\mathbf{a}^H(\theta) \mathbf{E}_n \mathbf{E}_n^H \mathbf{a}(\theta)}$
其中，$\mathbf{a}(\theta)$是给定方向$\theta$下的阵列流形向量，$\mathbf{E}_n$是噪声子空间的基底。

#### 5. 方向估计
通过搜索空间谱函数$P(\theta)$的峰值来估计信号源的方向。峰值的位置对应于信号源的方向。

### 特点
- MUSIC算法提供了非常高的分辨率，能够区分相距非常近的信号源。
- 它是一种基于数据的方法，不需要关于信号或噪声的先验知识。
- 适用于信号源数少于传感器数的情况。

### 限制
- 当信号源数目接近或超过传感器数目时，性能会下降。
- 对模型中的假设（如信号和噪声的统计独立性）敏感。

MUSIC算法是阵列信号处理中一个非常强大的工具，广泛应用于雷达、声纳、无线通信等领域。
